In [11]:
using Pkg
Pkg.resolve()
Pkg.clone("https://github.com/DeepWaterModels/FModels.jl")
Pkg.update()

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/DeepWaterModels/FModels.jl`


┌ Warning: Pkg.clone is only kept for legacy CI script reasons, please use `add`
└ @ Pkg.API /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:463


 Resolving package versions...


┌ Info: Path `/Users/navaro/.julia/dev/FModels` exists and looks like the correct package, using existing path instead of cloning
└ @ Pkg.Types /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Pkg/src/Types.jl:586


  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaVlasov/SplittingOperators.jl`
  Updating git-repo `https://github.com/pnavaro/FINUFFT.jl`
  Updating git-repo `https://github.com/pnavaro/Splittings.jl.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
using  FFTW
using  LinearAlgebra
using  ProgressMeter
using  BenchmarkTools
using  Plots
using  FModels

pyplot()

Plots.PyPlotBackend()

In [3]:
abstract type AbstractModel end

mutable struct Cheng <: AbstractModel

    freq    :: Freq
    label   :: String
    data    :: Vector{Tuple{Vector{Complex{Float64}},Vector{Complex{Float64}}}}
    Gamma   :: Array{Float64,1}
    Dx      :: Array{Complex{Float64},1}
    H       :: Array{Complex{Float64},1}
    Pi      :: BitArray{1}
    epsilon :: Float64
    hnew    :: Vector{Complex{Float64}}
    unew    :: Vector{Complex{Float64}}
    
    Int1    :: Vector{Complex{Float64}}
    Int2    :: Vector{Complex{Float64}}
    Int3    :: Vector{Complex{Float64}}
    
    Px      :: FFTW.FFTWPlan
        
    function Cheng(freq::Freq, epsilon::Float64)
        label = "Cheng et al."
        data  = []
        Gamma = abs.(freq.k)
        Dx    =  1im * freq.k            # Differentiation
        H     = -1im * sign.(freq.k)     # Hilbert transform
        Pi    = Gamma .< freq.kmax * 2/3 # Dealiasing low-pass filter
        
        hnew = zeros(Complex{Float64}, freq.nk)
        unew = zeros(Complex{Float64}, freq.nk)
    
        Int1 = zeros(Complex{Float64}, freq.nk)
        Int2 = zeros(Complex{Float64}, freq.nk)
        Int3 = zeros(Complex{Float64}, freq.nk)
        
        Px  = plan_fft(hnew; flags = FFTW.MEASURE)
      
        new(freq, label, data, Gamma, Dx, H, Pi, epsilon,
            hnew, unew, Int1, Int2, Int3, Px)
        
    end
end

function (m::Cheng)(h::Vector{Complex{Float64}},
                    u::Vector{Complex{Float64}})
         
         ldiv!(m.hnew, m.Px , h)
         
         m.Int1  .= u
         m.Int1 .*= Gamma
         ldiv!(m.unew, m.Px , m.Int1)
         m.unew  .^= 2
         mul!(m.Int1, m.Px , m.unew)
         m.Int1 .*= m.H
         
         m.Int2  .= m.h
         m.Int2 .*= m.Dx
         ldiv!(m.unew, m.Px , m.Int2)
         m.unew  .*= m.hnew
         mul!(m.Int2, m.Px , m.unew)
         
         m.Int3  .= h
         m.Int3 .*= m.Gamma
         ldiv!(m.unew, m.Px, m.Int3)
         m.unew  .*= m.hnew
         mul!(m.Int3 , m.Px , m.unew)
         m.Int3 .*= m.H
         
         m.hnew  .= -u
         m.hnew .*= m.Dx
         
         m.Int1 .-= m.Int2
         m.Int1 .-= m.Int3
         m.Int1 .*= m.Pi
         m.Int1 .*= m.epsilon
         
         u  .= h
         u .*= m.H
         u .+= m.Int1
         
         h .= m.hnew
         
    end
    
end



LoadError: syntax: unexpected "end"

In [4]:
mutable struct Matsuno <: AbstractModel
    
    freq    :: Freq
    label   :: String
    data    :: Vector{Tuple{Vector{Complex{Float64}},Vector{Complex{Float64}}}}
    Gamma   :: Array{Float64,1}
    Dx      :: Array{Complex{Float64},1}
    H       :: Array{Complex{Float64},1}
    Pi      :: BitArray{1}
    epsilon :: Float64
    hnew    :: Vector{Complex{Float64}}
    unew    :: Vector{Complex{Float64}}
    
    Int1    :: Vector{Complex{Float64}}
    Int2    :: Vector{Complex{Float64}}
    Int3    :: Vector{Complex{Float64}}
    
    Px      :: FFTW.FFTWPlan

    function Matsuno(freq::Freq, epsilon::Float64)
        label = "Matsuno"
        data  = []
        Gamma = abs.(freq.k)
        Dx    =  1im * freq.k            # Differentiation
        H     = -1im * sign.(freq.k)     # Hilbert transform
        Pi    = Gamma .< freq.kmax * 2/3 # Dealiasing low-pass filter
        hnew = zeros(Complex{Float64}, freq.nk)
        unew = zeros(Complex{Float64}, freq.nk)
    
        Int1 = zeros(Complex{Float64}, freq.nk)
        Int2 = zeros(Complex{Float64}, freq.nk)
        Int3 = zeros(Complex{Float64}, freq.nk)
        
        Px  = plan_fft(hnew; flags = FFTW.MEASURE)
      
        new(freq, label, data, Gamma, Dx, H, Pi, epsilon,
            hnew, unew, Int1, Int2, Int3, Px)
    end
end


function (m::Matsuno)(h::Vector{Complex{Float64}},
                      u::Vector{Complex{Float64}})
    
   # hnew .= real(ifft(h))
   # unew .= real(ifft(u))
   # Int3 .= fft(real(ifft(Dx.*h)).*real(ifft(Gamma.*h)))
 #
   # Int1 .= H.*u-epsilon*Pi.*(H.*fft(hnew.*real(ifft(Gamma.*u))).+Dx.*fft(hnew.*unew))
   # Int2 .= -(Dx.*h)-epsilon/2*Pi.*(Dx.*fft(unew.^2))+epsilon*Pi.*Int3
   # 
   # h .= Int1
   # u .= Int2
     
    m.hnew .= m.Gamma 
    m.hnew .*= h         
    ldiv!(m.unew, m.Px, m.hnew )   
    m.hnew .= m.Dx 
    m.hnew .*= m.h            
    ldiv!(m.Int1, m.Px, m.hnew)    
    m.unew .*= m.Int1            
    mul!(m.Int1, m.Px, m.unew)     
    m.Int1 .*= m.epsilon*m.Pi
    m.Int2 .= m.Dx .* h
    m.Int1 .-= m.Int2
    ldiv!(m.hnew, m.Px, h)
    ldiv!(m.unew, m.Px, u)
    m.Int2  .= m.hnew .* m.unew
    mul!(m.Int3, m.Px, m.Int2)
    m.Int3 .*= m.Dx
    h     .= m.H.*u
    u    .*= m.Gamma
    ldiv!(m.Int2, m.Px, u)
    m.Int2 .*= m.hnew
    mul!(u, m.Px, m.Int2)
    u    .*= m.H
    m.Int3 .+= u
    m.Int3 .*= epsilon * m.Pi
    h    .-= m.Int3     
    m.Int3  .= m.unew.^2
    mul!(m.unew, m.Px, m.Int3)
    m.unew  .*= m.Dx
    m.unew  .*= m.epsilon/2 * m.Pi 
    m.Int1  .-= m.unew
    u .= m.Int1
end
    

In [5]:
function solve!(model::AbstractModel, h, u, times::Times, solver::TimeSolver)
                
    prog = Progress(times.nt,1) # progress bar
    
    model.data = []
  
    push!(model.data,(h,u))
    for l in range(1,times.nt-1)
        
        dt = times.t[l+1]-times.t[l]
        
        step!( solver, model, h, u, dt)
    
        push!(model.data,(h,u))   
        next!(prog)
    end
            
end

solve! (generic function with 1 method)

In [6]:
function run_simulations(mesh::Mesh, models, times::Times)
   
    solver = RK4( mesh.nx )
    h  = zeros(Complex{Float64}, mesh.nx)
    u  = zeros(Complex{Float64}, mesh.nx)

    for model in models
            
        h .= exp.(-mesh.x.^2)
        h .= model.Pi .* fft(h)
        u .= model.Pi .* fft(u)
        
        solve!(model, h, u, times, solver )
    end
    
end

run_simulations (generic function with 1 method)

In [7]:
epsilon = 1/2
N       = 2^12
L       = 10
T       = 5
dt      = 0.001

@show epsilon,N,L,T,dt

mesh  = Mesh(-L, L, N)
freq  = Freq(mesh.dx, N)
times = Times(dt, T)

models = [Cheng(freq, epsilon), Matsuno(freq, epsilon)]
    
@time run_simulations(mesh, models, times)
fig(5, times, models, mesh)

(epsilon, N, L, T, dt) = (0.5, 4096, 10, 5, 0.001)


UndefVarError: UndefVarError: step! not defined